In [1]:
!pip install -q openai #--progress-bar off

In [2]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [ ]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = "xxx" # "" #input("INSERT YOU OPENAI API KEY:")
# clear_output()

In [4]:
output_datapath = "data/output/2023.11.16-1_Llama2_70B_API/"

In [5]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [6]:
data = pd.read_excel("../data/input/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx")
data = data.drop(columns=['Unnamed: 0', 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [7]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with the label only using the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

system_prompt = """In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

Answer with the label only using the format:---LABEL---
"""


In [8]:
data_sel = data #.sample(2)

In [9]:
import time

cnt = 0
results = []
for index, row in data_sel.iterrows():
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]
    
    #query = template_part1 + text + template_part2
    query = text
    
    body = {
      "model": "meta-llama/Llama-2-70b-chat-hf",
      "messages": [
          {"role": "system", "content": system_prompt}, 
          {"role": "user", "content": query}],
      "temperature": 0.0
    }
    
    result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
    
    print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    
    result_extended = (cnt, text_id, sdg_true, result)
    
    with open(output_datapath+f'sdg_classification_Llama2_13B_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    cnt+=1
    time.sleep(0.5)

0 6668 0   ---SDG 11---

The text reports on the resumption of subway service in Brooklyn after a train derailment, which suggests that the incident did not result in any serious injuries or fatalities. This news aligns with SDG 11, which aims to make cities and human settlements inclusive, safe, resilient, and sustainable. The report highlights the efforts of the authorities to restore service and ensure the safety of the public, which contributes to achieving this goal.
1 6099 0   ---SDG 8---

This text can be classified under SDG 8, which is "Decent Work and Economic Growth." The article mentions Brooklyn Decker's photo shoot in the Maldives, which is a popular tourist destination. The article highlights the beauty of the location and the photos, which suggests that it is promoting tourism and economic growth.
2 6471 0   ---SDG 16---

The article discusses the court case of Afroz Khan Pathan, who was convicted for conspiracy to strike terror and sentenced to life imprisonment. Howev

19 6073 0   ---SDG 0---

This text cannot be classified under any of the 17 Sustainable Development Goals (SDGs) adopted by the United Nations Member States in 2015, as it does not address any of the specific goals or targets mentioned in the SDGs. The text only discusses the MLB Power Rankings and does not relate to any of the sustainable development goals.
20 1376 0   ---SDG 13: Climate Action---
21 3889 0   ---SDG 8: Decent Work and Economic Growth---

The text highlights the performance of various cricket players and teams in the ICC World Cup 2023 qualifiers, which relates to SDG 8, Decent Work and Economic Growth, as it showcases the economic and social impact of sports on individuals and communities. The text also mentions the winning streak of Sri Lanka, which indicates the team's growth and success in the tournament, further relating to SDG 8.
22 6576 0   ---SDG 3: Good Health and Well-being---
23 3912 0   ---SDG 8: Decent Work and Economic Growth---

The article discusses the

43 4505 0   ---SDG 3: Good Health and Well-being---
44 1776 0   ---SDG 10: Reduced Inequalities---

The text mentions the Seattle Pride Parade, which is an event that promotes diversity and inclusion, and reduces inequalities for the LGBTQ+ community. Therefore, it aligns with SDG 10: Reduced Inequalities.
45 1529 0   ---SDG 9: Industry, Innovation, and Infrastructure---

This news article can be classified under SDG 9, as it relates to the development of innovative technologies and infrastructure in the field of chip manufacturing. The investment by a Japan-backed fund in a chip materials maker, JSR, aims to strengthen the country's position in the global chip supply chain, which aligns with the goals of SDG 9.
46 1838 0   ---SDG 8: Decent Work and Economic Growth---

The text mentions the transfer of football players and their contracts, which falls under the category of decent work and economic growth. The players are seeking to advance their careers and earn a living, while the clu

126 720 2   ---SDG 2: Zero Hunger---
127 6780 2   ---SDG 2: Zero Hunger---
128 2407 2   ---SDG 12: Responsible Consumption and Production---
129 3672 2   ---SDG 2: Zero Hunger---
130 4315 2   ---SDG 2: Zero Hunger---
131 3218 2   ---SDG 2: Zero Hunger---
132 6569 2   ---SDG 2: Zero Hunger---
133 98 2   ---SDG 2: Zero Hunger---
134 3973 2   ---SDG 2: Zero Hunger---
135 4844 2   ---SDG 2: Zero Hunger---
136 2970 2   ---SDG 2: Zero Hunger---
137 2747 2   ---SDG 2: Zero Hunger---
138 3976 2   ---SDG 2: Zero Hunger---
139 2988 2   ---SDG 2: Zero Hunger---
140 2775 2   ---SDG 2: Zero Hunger---
141 5423 2   ---SDG 8: Decent Work and Economic Growth---
142 5748 2   ---SDG 2: Zero Hunger---
143 1642 2   ---SDG 2: Zero Hunger---
144 2727 2   ---SDG 2: Zero Hunger---
145 869 2   ---SDG 2: Zero Hunger---
146 3205 2   ---SDG 2: Zero Hunger---
147 6506 2   ---SDG 2: Zero Hunger---
148 1622 2   ---SDG 2: Zero Hunger---
149 2827 2   ---SDG 8: Decent Work and Economic Growth---
150 223 2   ---SDG 2: Ze

299 3110 4   ---SDG 4: Quality Education---
300 4132 5   ---SDG 5: Gender Equality---
301 660 5   ---SDG 5: Gender Equality---
302 3168 5   ---SDG 5: Gender Equality---
303 5533 5   ---SDG 5---

This text can be classified under SDG 5, which is gender equality. The text highlights the lack of gender balance in judicial appointments and the low representation of women in senior positions in the judiciary. It also mentions efforts to implement quota requirements in the Federal Electoral Law to address this issue.
304 3397 5   ---SDG 5: Gender Equality---
305 6124 5   ---SDG 8: Decent Work and Economic Growth---
306 1514 5   ---SDG 5: Gender Equality---
307 124 5   ---SDG 5: Gender Equality---
308 5664 5   ---SDG 5: Gender Equality---
309 4769 5   ---SDG 5: Gender Equality---
310 18 5   ---SDG 5: Gender Equality---
311 5819 5   ---SDG 5: Gender Equality---
312 1266 5   ---SDG 5: Gender Equality---
313 3608 5   ---SDG 8: Decent Work and Economic Growth---
314 1094 5   ---SDG 4: Quality Edu

...

801 4843 13   ---SDG 13: Climate Action---
802 6675 13   ---SDG 17: Partnerships for the Goals---
803 6696 13   ---SDG 13: Climate Action---
804 5030 13   ---SDG 17: Partnerships for the Goals---
805 5047 13   ---SDG 13: Climate Action---
806 220 13   ---SDG 13: Climate Action---
807 1613 13   ---SDG 12: Responsible Consumption and Production---
808 1863 13   ---SDG 7: Affordable and Clean Energy---
809 5441 13   ---SDG 13: Climate Action---
810 3560 13   ---SDG 11: Sustainable Cities and Communities---
811 5575 13   ---SDG 4: Quality Education---
812 3518 13   ---SDG 13: Climate Action---
813 2094 13   ---SDG 17: Partnerships for the Goals---
814 5761 13   ---SDG 13: Climate Action---
815 4378 13   ---SDG 17: Partnerships for the Goals---
816 4186 13   ---SDG 13: Climate Action---
817 3475 13   ---SDG 13: Climate Action---
818 4235 13   ---SDG 13: Climate Action---
819 836 13   ---SDG 13: Climate Action---
820 4776 13   ---SDG 13: Climate Action---
821 2958 13   ---SDG 13: Climate Act

920 1767 15   ---SDG 8: Sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all.---
921 1957 15   ---SDG 13: Climate Action---
922 980 15   ---SDG 8---
923 1299 15   ---SDG 8---

The text highlights the need for international guidance, rules, and codes of conduct on legality, due diligence, and forest certification, which relates to SDG 8, Decent Work and Economic Growth. The focus on ensuring conformance with new regulations and the requirement for businesses to exercise increased due diligence in sourcing and selling wood and wood products also aligns with this SDG.
924 5980 15   ---SDG 15---

This text can be classified under SDG 15, Life on Land, as it discusses the classification of ecosystems and the availability of resources/services in relation to biodiversity.
925 2985 15   ---SDG 15---

This text can be classified under SDG 15, Life on Land, as it mentions the study of plant and animal species, including endemic and subende

972 6738 16   ---SDG 16: Peace, Justice and Strong Institutions---
973 3540 16   ---SDG 16---

The text mentions enslavement, imprisonment, enforced disappearance, torture, persecution, rape, murder, and other inhumane acts, which are all related to the theme of peace, justice, and strong institutions. Therefore, it falls under SDG 16, which aims to promote peaceful and inclusive societies, access to justice for all, and effective and accountable institutions at all levels.
974 283 16   ---SDG 16---

The text discusses the public enforcement regime in China and its efforts to deter securities crimes. The China Securities Regulatory Commission's 38 regional offices have increased their enforcement efforts since 2011, but the overall enforcement of securities law is still considered weak due to a lack of reliable private enforcement regime and limited monetary penalties. The text also highlights a pattern of selective enforcement, where private-owned listed firms face disproportionately 

994 5916 16   ---SDG 16---

This text can be classified under SDG 16, which is "Peace, Justice and Strong Institutions." The text mentions state crimes, human rights violations, and corruption, which are all issues that fall under the purview of this goal. The goal aims to promote peaceful and inclusive societies, access to justice, and effective and accountable institutions, which are all relevant to the issues mentioned in the text.
995 567 16   ---SDG 16---

This text can be classified under SDG 16, which is "Peace, Justice and Strong Institutions." The text discusses the role of governments in regulating the internet and their impact on individuals' privacy, freedom of expression, and freedom of opinion. It also mentions the potential for abuse of power and attacks on human rights, which falls under the purview of SDG 16.
996 4214 16   ---SDG 16: Peace, Justice and Strong Institutions---
997 322 16   ---SDG 8---

The text discusses the importance of human rights in the civilizing p

1013 6330 16   ---SDG 16---

The text discusses the principle of subsidiarity in the context of multi-level governance systems and its application in the judicial field, specifically in the context of human rights complaints. SDG 16, "Peace, Justice and Strong Institutions," aligns with the focus on the role of courts and the importance of subsidiarity in ensuring that decisions are made at the appropriate level to promote strong and effective institutions.
1014 2723 16   ---SDG 16: Peace, Justice and Strong Institutions---
1015 4695 16   ---SDG 16: Peace, Justice and Strong Institutions---
1016 6628 16   ---SDG 16---

This text can be classified under SDG 16, which is "Peace, Justice and Strong Institutions." The creation of global institutions, international treaties, and international norms mentioned in the text aligns with the goals of SDG 16, which aims to promote peaceful and inclusive societies, access to justice for all, and effective and accountable institutions at all levels.

In [10]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [11]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    if len(res[3]['choices'])>1:
        raise Exception('ERROR: Multilabel detected!')
    sdg_pred = res[3]['choices'][0]['message']['content']
    res4df = (res[0], res[1], res[2], sdg_pred)
    print(res4df)
    tuple_list.append(res4df)

(865, 2639, 14, '  ---SDG 14: Life Below Water---')
(531, 6741, 8, '  ---SDG 8: Decent Work and Economic Growth---')
(256, 1972, 4, '  ---SDG 4: Quality Education---')
(816, 4186, 13, '  ---SDG 13: Climate Action---')
(444, 2876, 7, '  ---SDG 7: Affordable and Clean Energy---')
(852, 5523, 14, '  ---SDG 14: Life Below Water---')
(690, 5806, 11, '  ---SDG 8: Decent Work and Economic Growth---')
(375, 3955, 6, '  ---SDG 6: Clean Water and Sanitation---')
(43, 4505, 0, '  ---SDG 3: Good Health and Well-being---')
(496, 1840, 8, '  ---SDG 8: Decent Work and Economic Growth---')
(466, 5293, 7, '  ---SDG 7: Affordable and Clean Energy---')
(605, 1052, 10, '  ---SDG 1: No Poverty---')
(932, 1672, 15, '  ---SDG 15: Life on Land---')
(906, 2788, 15, '  ---SDG 15: Life on Land---')
(235, 5055, 3, '  ---SDG 8---')
(40, 6153, 0, '  ---SDG 8: Decent Work and Economic Growth---')
(419, 2267, 6, '  ---SDG 6: Clean Water and Sanitation---')
(910, 1455, 15, '  ---SDG 15---')
(764, 4097, 12, '  ---SDG 1

In [12]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [13]:
results_processed

,cnt,id,true_label,predicted_label
0,865,2639,14,---SDG 14: Life Below Water---
1,531,6741,8,---SDG 8: Decent Work and Economic Growth---
2,256,1972,4,---SDG 4: Quality Education---
3,816,4186,13,---SDG 13: Climate Action---
4,444,2876,7,---SDG 7: Affordable and Clean Energy---
...,...,...,...,...
1015,487,433,8,---SDG 1: No Poverty---
1016,34,6228,0,---SDG 4: Quality Education---\n\nThe articl...
1017,37,4226,0,---SDG 0---\n\nThe text does not mention any...
1018,907,3261,15,---SDG 15---


In [14]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,---SDG 11---\n\nThe text reports on the resu...
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,---SDG 8---\n\nThis text can be classified u...
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 16---\n\nThe article discusses the co...
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,---SDG 4: Quality Education---\n\nThe text h...
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,---SDG 8: Decent Work and Economic Growth---...
...,...,...,...,...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"---SDG 16: Peace, Justice and Strong Institu..."
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,---SDG 16---\n\nThis text can be classified ...
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"---SDG 16: Peace, Justice and Strong Institu..."
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,---SDG 16---\n\nThe text discusses the legal...


In [15]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label,predicted_label_clean
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,---SDG 11---\n\nThe text reports on the resu...,11
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,---SDG 8---\n\nThis text can be classified u...,8
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 16---\n\nThe article discusses the co...,16
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,---SDG 4: Quality Education---\n\nThe text h...,4
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,---SDG 8: Decent Work and Economic Growth---...,8
...,...,...,...,...,...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"---SDG 16: Peace, Justice and Strong Institu...",16
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,---SDG 16---\n\nThis text can be classified ...,16
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"---SDG 16: Peace, Justice and Strong Institu...",16
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,---SDG 16---\n\nThe text discusses the legal...,16


In [16]:
results_processed_merged.groupby("predicted_label_clean").count()

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,
0,16,10,16,16,10,10,10,16,16,16,16
08,1,1,1,1,1,1,1,1,1,1,1
1,47,47,47,47,47,47,47,47,47,47,47
10,39,35,39,39,35,35,35,39,39,39,39
11,66,64,66,66,64,64,64,66,66,66,66
12,40,40,40,40,40,40,40,40,40,40,40
13,47,46,47,47,46,46,46,47,47,47,47
14,49,49,49,49,49,49,49,49,49,49,49
15,39,37,39,39,37,37,37,39,39,39,39


In [17]:
results_processed_merged.to_excel(output_datapath+'report_sdg_classification_Llama2_13B.xlsx')

In [18]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.38      0.10      0.16        60
           1       0.83      0.65      0.73        60
           2       0.94      0.80      0.86        60
           3       0.75      0.95      0.84        60
           4       0.75      0.93      0.83        60
           5       0.94      0.52      0.67        60
           6       0.97      0.95      0.96        60
           7       0.92      0.82      0.87        60
           8       0.23      0.85      0.36        60
           9       0.80      0.40      0.53        60
          10       0.59      0.38      0.46        60
          11       0.76      0.83      0.79        60
          12       0.90      0.60      0.72        60
          13       0.87      0.68      0.77        60
          14       0.98      0.80      0.88        60
          15       0.85      0.55      0.67        60
          16       0.83      0.83      0.83        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
